In [1]:
#Q1A
import polars as pl

URL = "https://raw.githubusercontent.com/philhetzel/opan5510-class12/refs/heads/main/data/economics_long.csv"

# Load and coerce types explicitly so pivot is predictable
economics_long = (
    pl.read_csv(URL, infer_schema_length=10000)
    .with_columns(
        pl.col("date").cast(pl.Utf8).str.strptime(pl.Date, strict=False),
        pl.col("variable").cast(pl.Categorical),
        pl.col("value").cast(pl.Float64),
    )
)

# Pivot: index=date, columns=variable, values=value
# Use aggregate_function="first" to guard against any rare duplicates
economics_wide = (
    economics_long
    .pivot(index="date", columns="variable", values="value", aggregate_function="first")
    .sort("date")
)

economics_wide


/tmp/ipython-input-3530322667.py:20: DeprecationWarning: The argument `columns` for `DataFrame.pivot` is deprecated. It has been renamed to `on`.
  .pivot(index="date", columns="variable", values="value", aggregate_function="first")


date,pce,pop,psavert,uempmed,unemploy
date,f64,f64,f64,f64,f64
1967-07-01,506.7,198712.0,12.6,4.5,2944.0
1967-08-01,509.8,198911.0,12.6,4.7,2945.0
1967-09-01,515.6,199113.0,11.9,4.6,2958.0
1967-10-01,512.2,199311.0,12.9,4.9,3143.0
1967-11-01,517.4,199498.0,12.8,4.7,3066.0
…,…,…,…,…,…
2014-12-01,12062.0,319746.157,7.6,12.9,8717.0
2015-01-01,12046.0,319928.646,7.7,13.2,8903.0
2015-02-01,12082.4,320074.511,7.9,12.9,8610.0


In [2]:
#Q1B
import polars as pl
from datetime import date

# Filter for dates >= 2010-01-01 and compute the median of 'unemploy'
cutoff = date(2010, 1, 1)

median_df = (
    economics_wide
    .filter(pl.col("date") >= cutoff)
    .select(
        pl.col("unemploy").cast(pl.Float64).median().alias("median_unemploy")
    )
)

median_unemploy = median_df["median_unemploy"][0]
print(f"The median unemployment for 2010-01-01 and beyond is: {median_unemploy}")

The median unemployment for 2010-01-01 and beyond is: 12471.0


In [5]:
#Q2A
import polars as pl

URL = "https://raw.githubusercontent.com/philhetzel/opan5510-class12/refs/heads/main/data/AirPassengers.txt"

# Load dataset
air_passengers = pl.read_csv(
    URL,
    separator="\t",
    infer_schema_length=5000,
    null_values=["", "NA", "N/A"]
).with_columns(
    pl.col("Year").cast(pl.Int32)
)

print("Original data shape:", air_passengers.shape)
print("\nFirst few rows:")
print(air_passengers.head())

# Month columns = everything except 'Year'
month_cols = [c for c in air_passengers.columns if c != "Year"]

# Melt to long form and normalize types (robust whitespace trim via regex)
air_passengers_long = (
    air_passengers
    .melt(
        id_vars="Year",
        value_vars=month_cols,
        variable_name="month",
        value_name="passengers"
    )
    .with_columns(
        pl.col("month").cast(pl.Utf8).str.replace_all(r"^\s+|\s+$", "").alias("month"),
        pl.col("passengers").cast(pl.Int64).alias("passengers")
    )
)

print("\nUnpivoted DataFrame:")
print(air_passengers_long.head(15))


Original data shape: (12, 13)

First few rows:
shape: (5, 13)
┌──────┬─────┬─────┬─────┬───┬─────┬─────┬─────┬─────┐
│ Year ┆ Jan ┆ Feb ┆ Mar ┆ … ┆ Sep ┆ Oct ┆ Nov ┆ Dec │
│ ---  ┆ --- ┆ --- ┆ --- ┆   ┆ --- ┆ --- ┆ --- ┆ --- │
│ i32  ┆ i64 ┆ i64 ┆ i64 ┆   ┆ i64 ┆ i64 ┆ i64 ┆ i64 │
╞══════╪═════╪═════╪═════╪═══╪═════╪═════╪═════╪═════╡
│ 1949 ┆ 112 ┆ 118 ┆ 132 ┆ … ┆ 136 ┆ 119 ┆ 104 ┆ 118 │
│ 1950 ┆ 115 ┆ 126 ┆ 141 ┆ … ┆ 158 ┆ 133 ┆ 114 ┆ 140 │
│ 1951 ┆ 145 ┆ 150 ┆ 178 ┆ … ┆ 184 ┆ 162 ┆ 146 ┆ 166 │
│ 1952 ┆ 171 ┆ 180 ┆ 193 ┆ … ┆ 209 ┆ 191 ┆ 172 ┆ 194 │
│ 1953 ┆ 196 ┆ 196 ┆ 236 ┆ … ┆ 237 ┆ 211 ┆ 180 ┆ 201 │
└──────┴─────┴─────┴─────┴───┴─────┴─────┴─────┴─────┘

Unpivoted DataFrame:
shape: (15, 3)
┌──────┬───────┬────────────┐
│ Year ┆ month ┆ passengers │
│ ---  ┆ ---   ┆ ---        │
│ i32  ┆ str   ┆ i64        │
╞══════╪═══════╪════════════╡
│ 1949 ┆ Jan   ┆ 112        │
│ 1950 ┆ Jan   ┆ 115        │
│ 1951 ┆ Jan   ┆ 145        │
│ 1952 ┆ Jan   ┆ 171        │
│ 1953 ┆ Jan   ┆ 196      

/tmp/ipython-input-2195259839.py:26: DeprecationWarning: `DataFrame.melt` is deprecated. Use `unpivot` instead, with `index` instead of `id_vars` and `on` instead of `value_vars`
  .melt(


In [6]:
#Q2B
import polars as pl

# Filter for 1955–1960 and compute the standard deviation of passengers
std_passengers = (
    air_passengers_long
    .filter((pl.col("Year") >= 1955) & (pl.col("Year") <= 1960))
    .select(pl.col("passengers").cast(pl.Float64).std().alias("std_passengers"))
    .to_series()          # extract the single column as a Series
    .item()               # get the single scalar
)

print(f"The standard deviation of airline passengers between 1955 and 1960 is: {std_passengers:.2f}")


The standard deviation of airline passengers between 1955 and 1960 is: 86.44


In [7]:
#Q3A
import polars as pl

URL = "https://raw.githubusercontent.com/philhetzel/opan5510-class12/refs/heads/main/data/table2.csv"

# Load with explicit types for stability
table2 = (
    pl.read_csv(URL, infer_schema_length=1000)
    .with_columns(
        pl.col("country").cast(pl.Utf8),
        pl.col("year").cast(pl.Int32),
        pl.col("type").cast(pl.Categorical),
        pl.col("count").cast(pl.Int64),
    )
)

print("Original data:")
print(table2)

# Pivot: rows by (country, year); columns from 'type'; values from 'count'
# Use aggregate_function='first' to handle any accidental duplicates safely
table2_wide = (
    table2
    .pivot(
        index=["country", "year"],
        columns="type",
        values="count",
        aggregate_function="first",
    )
    .sort(["country", "year"])
)

print("\nPivoted DataFrame:")
print(table2_wide)

Original data:
shape: (12, 5)
┌──────────┬─────────────┬──────┬────────────┬────────────┐
│ rownames ┆ country     ┆ year ┆ type       ┆ count      │
│ ---      ┆ ---         ┆ ---  ┆ ---        ┆ ---        │
│ i64      ┆ str         ┆ i32  ┆ cat        ┆ i64        │
╞══════════╪═════════════╪══════╪════════════╪════════════╡
│ 1        ┆ Afghanistan ┆ 1999 ┆ cases      ┆ 745        │
│ 2        ┆ Afghanistan ┆ 1999 ┆ population ┆ 19987071   │
│ 3        ┆ Afghanistan ┆ 2000 ┆ cases      ┆ 2666       │
│ 4        ┆ Afghanistan ┆ 2000 ┆ population ┆ 20595360   │
│ 5        ┆ Brazil      ┆ 1999 ┆ cases      ┆ 37737      │
│ …        ┆ …           ┆ …    ┆ …          ┆ …          │
│ 8        ┆ Brazil      ┆ 2000 ┆ population ┆ 174504898  │
│ 9        ┆ China       ┆ 1999 ┆ cases      ┆ 212258     │
│ 10       ┆ China       ┆ 1999 ┆ population ┆ 1272915272 │
│ 11       ┆ China       ┆ 2000 ┆ cases      ┆ 213766     │
│ 12       ┆ China       ┆ 2000 ┆ population ┆ 1280428583 │
└─────────

/tmp/ipython-input-17413536.py:24: DeprecationWarning: The argument `columns` for `DataFrame.pivot` is deprecated. It has been renamed to `on`.
  .pivot(


In [8]:
#Q3B
import polars as pl

# 1) Add a 'ratio' column = cases / population (guard against zeros / nulls; cast to float)
table2_with_ratio = (
    table2_wide
    .with_columns(
        pl.when(pl.col("population").cast(pl.Float64) > 0)
          .then(pl.col("cases").cast(pl.Float64) / pl.col("population").cast(pl.Float64))
          .otherwise(None)
          .alias("ratio")
    )
)

# 2) Filter for year 1999
year_1999 = table2_with_ratio.filter(pl.col("year") == 1999)

# 3) Find country with highest ratio
top_country_df = (
    year_1999
    .sort("ratio", descending=True, nulls_last=True)
    .select("country")
    .head(1)
)

highest_ratio_country = top_country_df["country"][0] if top_country_df.height > 0 else None

print("Data for 1999 with ratios:")
print(year_1999)

print(f"\nThe country with the highest cases/population ratio in 1999 is: {highest_ratio_country}")

Data for 1999 with ratios:
shape: (3, 5)
┌─────────────┬──────┬────────┬────────────┬──────────┐
│ country     ┆ year ┆ cases  ┆ population ┆ ratio    │
│ ---         ┆ ---  ┆ ---    ┆ ---        ┆ ---      │
│ str         ┆ i32  ┆ i64    ┆ i64        ┆ f64      │
╞═════════════╪══════╪════════╪════════════╪══════════╡
│ Afghanistan ┆ 1999 ┆ 745    ┆ 19987071   ┆ 0.000037 │
│ Brazil      ┆ 1999 ┆ 37737  ┆ 172006362  ┆ 0.000219 │
│ China       ┆ 1999 ┆ 212258 ┆ 1272915272 ┆ 0.000167 │
└─────────────┴──────┴────────┴────────────┴──────────┘

The country with the highest cases/population ratio in 1999 is: Brazil
